## Pareto Municipality

The dengue data is originally provided at the municipality level, which is the smallest administrative unit in Brazil.
Yet, Brazil is a very large country, having 5570 municipalities in the dataset. Creating forecasts for each
municipality would be computationally expensive and not very useful, as many of them have very few cases.
Thus, the goal of this notebook is to try to select a subset of municipalities.

#### Notes on Methodology

The Mosqlimate sprint objective is to predict on the state level. Therefore, our selection of municipalities must
consider such division.

Some states have much more concentration than others. Hence, we must determine a different quantity of  municipalities
to be used by UF.

- DF has a single municipality, so nothing has to be done.

In [1]:
import numpy as np
import pandas as pd
import polars as pl
pl.Config.set_tbl_rows(30)
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from lets_plot import *
LetsPlot.setup_html()


from inequality import gini

In [2]:
data = pl.read_parquet('../data/dataset_complete_dengue_municipality.parquet')

In [39]:
data = data.with_columns(np.log1p(pl.col('casos')).alias('log_casos'))

In [42]:
gdf = data.group_by(['geocode','uf']).agg(pl.col('casos').sum().alias('casos'))
gdf = gdf.join(gdf.group_by(['uf']).agg(pl.col('casos').sum().alias('casos_uf')),on=['uf'])
gdf = gdf.with_columns(
    np.log1p(pl.col('casos')).alias('log_casos'),
    np.log1p(pl.col('casos_uf')).alias('log_casos_uf'),
    (pl.col('casos')/pl.col('casos_uf')).alias('casos_percent_uf'),
)
gdf = gdf.with_columns((pl.col('log_casos')/pl.col('log_casos_uf')).alias('log_casos_percent_uf'))

gdf = gdf.with_columns(
    pl.col("casos_percent_uf")
    .rank("ordinal", descending=True)
    .over(["uf"])
    .alias("rank_within_uf")
)

gdf = gdf.sort(['uf','casos_percent_uf'],descending=True).with_columns(
    pl.col("casos_percent_uf")
    .cum_sum()
    .over(["uf"])
    .alias("cumulative_share")
)

gdf

geocode,uf,casos,casos_uf,log_casos,log_casos_uf,casos_percent_uf,log_casos_percent_uf,rank_within_uf,cumulative_share
i64,str,i64,i64,f64,f64,f64,f64,u32,f64
1721000,"""TO""",51706,126949,10.853348,11.751549,0.407297,0.923568,1,0.407297
1702109,"""TO""",14370,126949,9.572968,11.751549,0.113195,0.814613,2,0.520492
1718204,"""TO""",5152,126949,8.547334,11.751549,0.040583,0.727337,3,0.561076
1716109,"""TO""",4995,126949,8.516393,11.751549,0.039347,0.724704,4,0.600422
1709500,"""TO""",4238,126949,8.352083,11.751549,0.033383,0.710722,5,0.633806
1707009,"""TO""",2675,126949,7.892078,11.751549,0.021071,0.671578,6,0.654877
1705508,"""TO""",2403,126949,7.784889,11.751549,0.018929,0.662456,7,0.673806
1702000,"""TO""",2207,126949,7.699842,11.751549,0.017385,0.655219,8,0.691191
1721208,"""TO""",2200,126949,7.696667,11.751549,0.01733,0.654949,9,0.708521


In [6]:
from concentrationMetrics import Index
indices = Index()

concentration = gdf.group_by('uf').map_groups(
    lambda df: pl.DataFrame({
        'uf': [df[0, 'uf']],
        'gini': [indices.gini(df['casos'].to_numpy())],
        'hhi': [indices.hhi(df['casos'].to_numpy())]
    })
)

In [11]:
(
    ggplot(data=concentration)
    + geom_point(
        mapping=aes(x='uf',y='gini'),
        fill='blue',
        alpha=0.5
    )
)

In [68]:

# logit(0.9)

In [76]:
# (
#     ggplot(data=gdf)
#     + geom_histogram(
#         mapping=aes(x='cumulative_share',fill='uf'),
#         fill='blue',
#         alpha=0.5
#     )
#     + scale_x_log10()
#     + facet_grid(y='uf')
# )
def logit(x):
    if x == 0:
        return -10
    elif x == 1:
        return 10
    return np.log(x / (1 - x))

gdf = gdf.with_columns(
    pl.col('casos_percent_uf').map_elements(logit, return_dtype=float).alias('logit'),
)

In [80]:
(
    ggplot(data=gdf)
    + geom_density(
        mapping=aes(x='log_casos',fill='uf'),
        fill='blue',
        alpha=0.5
    )
    + facet_grid(y='uf')
)

In [87]:
(
    ggplot(data=gdf)
    + geom_density(
        mapping=aes(x='logit',fill='uf',color='uf'),
        alpha=0.5
    )
    + facet_grid(y='uf')
)

In [13]:
# gdf = data.group_by(['year','geocode','uf']).agg(pl.col('casos').sum().alias('casos'))
# gdf = gdf.join(gdf.group_by(['year','uf']).agg(pl.col('casos').sum().alias('casos_uf')),on=['uf','year'])
# gdf = gdf.with_columns(
#     np.log1p(pl.col('casos')).alias('log_casos'),
#     (pl.col('casos')/pl.col('casos_uf')).alias('casos_percent_uf')
# )
# gdf = gdf.with_columns(
#     pl.col("casos_percent_uf")
#     .rank("ordinal", descending=True)
#     .over(["uf", "year"])
#     .alias("rank_within_uf_year")
# )

# gdf = gdf.sort(['uf','year','casos_percent_uf'],descending=True).with_columns(
#     pl.col("casos_percent_uf")
#     .cum_sum()
#     .over(["uf", "year"])
#     .alias("cumulative_share")
# )

# top_municipalities = gdf.filter(pl.col("cumulative_share") <= 0.8)

In [11]:
# gdf['uf'].unique()

In [14]:
gdf.group_by("uf").map_groups(
    lambda df: df.with_columns(
        (pl.col("casos") / pl.col("casos").sum()).alias("share"),
    ).with_columns(
        (pl.col("share") ** 2).alias("share_squared")
    ).select([
        pl.first("uf"),
        pl.sum("share_squared").alias("hhi")  # Herfindahl Index
    ])
    )

uf,hhi
str,f64
"""TO""",0.186042
"""SP""",0.031962
"""SE""",0.245432
"""SC""",0.127813
"""RS""",0.060383
"""RR""",0.341974
"""RO""",0.05641
"""RN""",0.199417
"""RJ""",0.195325


In [15]:
gdf.group_by("uf").agg(
    pl.len().alias("num_municipalities"),
    pl.col("casos").kurtosis().alias("kurtosis")
).sort("kurtosis", descending=True)

uf,num_municipalities,kurtosis
str,u32,f64
"""DF""",1,NaN
"""MG""",853,530.055731
"""RS""",497,293.921423
"""SP""",645,281.936182
"""PI""",224,212.280571
"""PB""",223,202.212555
"""CE""",184,174.156177
"""GO""",246,170.496063
"""RN""",167,150.297201


In [16]:
# concentration_per_uf.filter(pl.col('len')< 30).sort('len',descending=True)

In [52]:
(
    ggplot(data=gdf)
    + geom_density(
        mapping=aes(x='uf', y='cumulative_share'),
    )
)

In [53]:
# (
#     ggplot(data=gdf)
#     + geom_point(
#         mapping=aes(x='uf', y='cumulative_share'),
#     )
# )

(
    ggplot(data=gdf)
    + geom_boxplot(
        mapping=aes(x='uf', y='cumulative_share'),
    )
)

In [ ]:
# gdf
# gdf.filter(pl.col('casos_percent_uf') > 0.05).group_by('uf').len()
# gdf.filter(pl.col('uf')=='CE')
# gdf.filter(pl.col('casos_percent_uf') > 0.05).group_by('uf').len()
# gdf.filter(pl.col('cumulative_share') < 0.80).group_by('uf').len().sort('uf')
concentration_per_uf = []
for i in np.linspace(0,1,5):
    conc = gdf.filter(pl.col('cumulative_share') < i).group_by('uf').len().sort('uf')
    concentration_per_uf.append(conc.with_columns(pl.lit(i).alias('cumulative_filter')))

# Concatenate a list of Polars DataFrames into a single DataFrame
concentration_per_uf = pl.concat(concentration_per_uf)



# gdf.filter(pl.col('cumulative_share') < 0.80)['uf'].count()
# (
#     ggplot(data=concentration_per_uf)
#     + geom_point(
#         mapping=aes(x='uf', y='len'),
#     )
# )

(
    ggplot(data=concentration_per_uf)
    + geom_point(
        mapping=aes(x='uf', y='len', color='cumulative_filter'),
    )
    + facet_grid(x='cumulative_filter')
)

In [ ]:
# gdf.filter(pl.col('uf')=='CE')
# gdf.filter(pl.col(''))
# gdf.filter(pl.col('casos_percent_uf') > 0.05)['uf'].unique()
# gdf.filter(pl.col('casos_percent_uf') > 0.05)['uf'].count()

# gdf.filter(pl.col('casos_percent_uf') > 0.01).group_by(['uf']).agg(
#     pl.col('casos_percent_uf').sum()
# )

# gdf.filter(pl.col('cumulative_share') < 0.99).group_by('uf').len().sort('uf')
# gdf.filter(pl.col('cumulative_share') < 0.99).group_by('uf').len().sort('uf')
# (gdf.group_by(['uf']).len()).join(gdf.filter(pl.col('cumulative_share') < 0.99).group_by('uf').len().sort('uf'),on='uf',how='left')

In [ ]:
partial = gdf.filter(pl.col('uf')=='BA',pl.col('rank_within_uf')<2)['casos'].sum()
total = gdf.filter(pl.col('uf')=='BA')['casos'].sum()

partial/total

In [ ]:
(
    # ggplot(data=gdf.filter(pl.col('uf')=='BA'))
    ggplot(data=gdf)
    + geom_point(
        mapping=aes(x='rank_within_uf',y='casos')
    )
    + facet_grid(y='uf',scales='free')
)

In [135]:
# gdf.filter(pl.col('uf')=='BA')

geocode,uf,casos,casos_uf,log_casos,casos_percent_uf,rank_within_uf,cumulative_share
i64,str,i64,i64,f64,f64,u32,f64
2927408,"""BA""",86207,870401,11.364518,0.099043,1,0.099043
2933307,"""BA""",52572,870401,10.869958,0.0604,2,0.159443
2914802,"""BA""",45972,870401,10.73581,0.052817,3,0.21226
2910800,"""BA""",42939,870401,10.667559,0.049332,4,0.261592
2913606,"""BA""",27071,870401,10.206255,0.031102,5,0.292694
2903201,"""BA""",18091,870401,9.803225,0.020785,6,0.313479
2931350,"""BA""",16314,870401,9.69984,0.018743,7,0.332222
2918001,"""BA""",14618,870401,9.590077,0.016795,8,0.349016
2930501,"""BA""",12349,870401,9.421411,0.014188,9,0.363204


In [123]:
(
    ggplot(data=gdf.filter(pl.col('uf')=='BA'))
    + geom_histogram(
        mapping=aes(x='casos_percent_uf')
    )
)

In [122]:
gdf

geocode,uf,casos,casos_uf,log_casos,casos_percent_uf,rank_within_uf_year,cumulative_share
i64,str,i64,i64,f64,f64,u32,f64
1721000,"""TO""",51706,126949,10.853348,0.407297,1,0.407297
1702109,"""TO""",14370,126949,9.572968,0.113195,2,0.520492
1718204,"""TO""",5152,126949,8.547334,0.040583,3,0.561076
1716109,"""TO""",4995,126949,8.516393,0.039347,4,0.600422
1709500,"""TO""",4238,126949,8.352083,0.033383,5,0.633806
1707009,"""TO""",2675,126949,7.892078,0.021071,6,0.654877
1705508,"""TO""",2403,126949,7.784889,0.018929,7,0.673806
1702000,"""TO""",2207,126949,7.699842,0.017385,8,0.691191
1721208,"""TO""",2200,126949,7.696667,0.01733,9,0.708521


In [113]:
gdf.filter(pl.col('uf')=='SP')

geocode,uf,casos,casos_uf,log_casos,casos_percent_uf,rank_within_uf_year,cumulative_share
i64,str,i64,i64,f64,f64,u32,f64
3550308,"""SP""",900240,6336613,13.710418,0.14207,1,0.14207
3509502,"""SP""",377486,6336613,12.841291,0.059572,2,0.201642
3549805,"""SP""",307529,6336613,12.636328,0.048532,3,0.250174
3543402,"""SP""",241705,6336613,12.395477,0.038144,4,0.288318
3552205,"""SP""",142431,6336613,11.86662,0.022477,5,0.310796
3549904,"""SP""",140386,6336613,11.852158,0.022155,6,0.33295
3518800,"""SP""",121837,6336613,11.710448,0.019227,7,0.352178
3541406,"""SP""",114469,6336613,11.648068,0.018065,8,0.370242
3506003,"""SP""",104302,6336613,11.555055,0.01646,9,0.386703


In [46]:
# Count how many years each municipality is in the top list
stability = top_municipalities.filter(pl.col('year')>2015, pl.col('year')<2025).group_by(['uf','geocode']).agg(
    pl.col('year').count().alias('years_in_top'),
    pl.col('casos').mean().alias('median_casos'),
    # (pl.col('years_in_top')/15).alias('stability'),
)

stability = stability.with_columns((pl.col('years_in_top')/10).alias('stability'))

In [56]:
# stability.filter(pl.col('stability')>0.4)
# stability
# top_municipalities['year'].unique()

In [10]:
# total_years_per_uf

In [8]:
# stability


In [33]:
# top_municipalities.filter(pl.col('uf')=='CE',pl.col('year')==2021).sort('casos')

In [34]:
# gdf.filter(pl.col('uf')=='CE',pl.col('year')==2020).sort('casos')

In [35]:
# gdf.filter(pl.col('year')==2020).group_by('uf').agg(
#     pl.col('casos').sum()
# )

In [38]:
# stability.filter(pl.col('uf')=='CE').sort('years_in_top').tail(10)

In [25]:
# gdf
# gdf.filter(pl.col('uf')=='CE',pl.col('year')==2021).sort('casos').sort('casos_percent_uf').with_columns(
# gdf.filter(pl.col('uf')=='CE',pl.col('year')==2021).with_columns(
#     pl.col('casos_percent_uf').sort()
#         # .cum_sum()
#         # .over(['uf','year'])
#         .alias('test')
# )

In [26]:

# Filter to those that cumulatively explain <= 80% of cases
# top_municipalities = gdf.filter(pl.col("cumulative_share") <= 0.8)

In [27]:
# data['geocode'].unique()

In [56]:
(
    ggplot(data=gdf.filter(pl.col('uf')=='CE'))
    + geom_histogram(
        mapping=aes(x='percent_uf')
    )
    + facet_grid(y='year')
)

In [50]:
(
    ggplot(data=gdf.filter(pl.col('uf')=='CE'))
    + geom_point(
        mapping=aes(x='geocode', y='log_casos'),
    )
    + facet_grid(y='year')
)